<center><a href="https://www.pieriantraining.com/" ><img src="../PTCenteredPurple.png" alt="Pierian Training Logo" /></a></center>


# Knowledge Retrieval - Single File in a Message

Let's break down the concept of Knowledge Retrieval in a more educational and explanatory tone:

**What is Knowledge Retrieval?**
Knowledge Retrieval is a feature that enhances the capabilities of an AI Assistant by integrating external knowledge sources. This can include specific information related to a product or documents provided by users.

**How Does it Work?**
When you upload a document to the AI Assistant, OpenAI processes this document in several steps:

1. **Chunking and Indexing**: The document is divided into smaller parts ("chunks") and indexed, which means it's organized in a way that the AI can search through it efficiently.

2. **Storing Embeddings**: Embeddings are like a condensed representation of the document's content, capturing its essential information in a format the AI can understand and retrieve when needed.

3. **Implementing Vector Search**: This is a method the AI uses to find the most relevant parts of your document based on the query it receives.

**Enabling Retrieval for an Assistant**
When you activate retrieval for a particular AI Assistant, all attached files are automatically processed as described above. However, this feature comes with a cost of $0.20 per gigabyte per day for each assistant that has retrieval enabled.

**Modifying Assistant Settings**
You can turn the retrieval feature on or off by using the Modify Assistant endpoint, which is a part of the Assistant's API.

**Retrieval Techniques**
The AI Assistant chooses between two retrieval techniques based on the user's messages:

1. **Direct Prompt Inclusion**: For short documents, the AI includes the content directly in its response prompt.

2. **Vector Search**: For longer documents, it performs a vector search to find the most relevant information.

 
Currently, the system prioritizes quality by including all relevant content in the model's context for answering queries. However, OpenAI plans to introduce more strategies for retrieval, allowing developers to choose a balance between the quality of retrieval and the cost of using the model. 
 

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI()

----

## Uploading Files

In [3]:
import os

In [4]:
files_to_upload = ['Wonka Chocolate Facility Rules.pdf']

In [5]:
def upload_assistant_file(filename):
    file = client.files.create(
      file=open(filename, "rb"),
      purpose='assistants'
    )
    print(file.id)

In [6]:
for file in files_to_upload:
    upload_assistant_file(file)

file-Dfd5QdbkpMuG7uXFZRVJ39OF


In [8]:
for file in client.files.list():
    print(f"{file.filename} has id of: {file.id}")

Wonka Chocolate Facility Rules.pdf has id of: file-Dfd5QdbkpMuG7uXFZRVJ39OF


## Adding Files for the Assistant (or in a message)

Let's explain the process of uploading files for retrieval in a more educational and explanatory way:

**Uploading Files for Retrieval: An Overview**
When you want to use external documents to enhance the AI Assistant's responses, you can upload these files for retrieval. This process is somewhat similar to how the Code Interpreter works.

**Two Levels of Attachment**

1. **Assistant-Level Attachment**: When you attach a file at this level, it becomes a part of the Assistant's general knowledge base. This means the Assistant can access this information when responding to any query.

2. **Message-Level Attachment**: In contrast, when you attach a file at the Message-level, it's only accessible for the specific conversation thread (or 'Thread') that the message is a part of. This is more targeted, as the file's content will only be used to respond to queries within that particular thread.

**How to Attach a File to a Message**
After uploading a file, you will receive a unique ID for that file. You can then use this ID to attach the file to a specific message. This way, the AI knows which document to refer to when crafting its response.

**Charges for File Usage**
It's important to note that the cost isn't based on the size of the files you upload. Instead, the charges are based on which files you attach to an Assistant or a Message that then get indexed (processed and made searchable by the AI).

**File Size and Format Limitations**
There are some limits on what you can upload:

- **Maximum File Size**: Each file can be up to 512 MB.
- **Token Limit**: The content of the file should not exceed 2,000,000 tokens (a token is roughly a word or a piece of a word, and this limit is calculated automatically when you attach the file).
- **Supported Formats**: The retrieval system is compatible with various file formats, including popular ones like PDF (.pdf), Markdown (.md), and Word documents (.docx).

In summary, by uploading files at either the Assistant-level or Message-level, you can significantly enhance the AI Assistant's ability to provide detailed and relevant responses based on the content of these documents. This feature supports various file formats and sizes, with costs incurred based on the indexing of attached files.

In [9]:
# Add the file to the assistant
assistant = client.beta.assistants.create(
    name='Rules Explainer',
  instructions="You answer information about rules based on your knowledge base of PDF files.",
  model="gpt-3.5-turbo-1106", # CHECK FOR ERROR IF YOU PROVIDE THE WRONG MODEL
  tools=[{"type": "retrieval"}],)

## Run a Thread and Add a File to Message

When a file is attached at the Message-level, it is only accessible within the specific Thread the Message is attached to. After having uploaded a file, you can pass the ID of this File when creating the Message. Note that you are not charged based on the size of the files you upload via the Files API but rather based on which files you attach to a specific Assistant or Message that get indexed.

In [34]:
thread = client.beta.threads.create()

In [35]:
file.id

'file-Dfd5QdbkpMuG7uXFZRVJ39OF'

In [36]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Can I bring my cat to the Wonka Chocolate Facility?",
    file_ids=[file.id],
    
)

In [37]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions = f"Use file with if: {file.id} to answer questions." # This helps to force the model to access the file!
)

In [38]:
run.status

'queued'

In [41]:
run = client.beta.threads.runs.retrieve(
          thread_id=thread.id,
          run_id=run.id
        )
    
print(run.status)

completed


Once the run is completed, let's get the messages

In [42]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc",
)

In [43]:
def display_thread_messages(messages):
    # EXPECTS MESSAGES IN ASC ORDER!
    for thread_message in messages.data:
        print(thread_message.content[0].text.value)
        print('\n')

In [44]:
display_thread_messages(messages)

Can I bring my cat to the Wonka Chocolate Facility?


I will start by opening the file to locate the relevant information about bringing a cat to the Wonka Chocolate Facility.


According to the facility rules, only guide dogs are allowed on the premises of the Wonka Chocolate Facility. Therefore, other pets, including cats, are not permitted at the facility【8†source】.




# Optional: Delete Assistant and All Files

In [45]:
my_assistants = client.beta.assistants.list(
    order="desc",
    limit="20",
)
response = client.beta.assistants.delete(my_assistants.data[0].id)
print(response)

AssistantDeleted(id='asst_ha6KdfCBxhfOciGI32qQeJnR', deleted=True, object='assistant.deleted')


In [48]:
for file in client.files.list():
    client.files.delete(file.id)

In [49]:
client.files.list()

SyncPage[FileObject](data=[], object='list', has_more=False)